# E. Build a database

You will build a database to hold your data. It is up to you to define appropriate tables and well as primary keys for connecting them. (The focus is definitely more on just using the basic methods/tools introduced in the course and not on building a complex database.) In particular, you can follow the following list of steps:

* restrict your data to 1000 entries and 5 columns of your choice
* create a connection to a sqlite3 database
* create one or multiple tables, at least one of the tables should have a PRIMARY KEY
* fill the database with your data
* run at least one query to demonstrate that it works correctly

In [8]:
# import the data that seems to be tab seperated
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3

In [9]:
pd.set_option('display.max_columns', 200)

In [10]:
# We will work on a subset of the columns
columns_product = [
    'code', 'product_name', 'countries_en', 'serving_size', 'energy_100g', 'nutrition_score_uk_100g'
]

columns_nutrient = ['code', 'product_name', 'proteins_100g', 'fat_100g', 'cholesterol_100g','carbohydrates_100g','sugars_100g' ]

# Load the DataFrame, we will work on the first 1000 rows and limit to 5 columns
#df=pd.read_csv("C:/Users/tgdkujo2/Documents/EPFL_Data_Analytics/2 - APPLIED DATA ANALYSIS/09. Course Project/world-food-facts/en.openfoodfacts.org.products.tsv",  nrows=1000, usecols=columns, sep="\t", low_memory=False)
df=pd.read_csv("C:/Users/tgdkujo2/Documents/EPFL_Data_Analytics/2 - APPLIED DATA ANALYSIS/09. Course Project/world-food-facts/en.openfoodfacts.org.products.tsv",  sep="\t", low_memory=False)

#df.set_index('code', inplace=True)
# rename columns that contain a "-", because they cause a problem in the database
df = df.rename(columns={'nutrition-score-uk_100g': 'nutrition_score_uk_100g'})

**Create an empty Database**

In [11]:
db = sqlite3.connect("my_database.db")

**Create Tables**

In [12]:
query_product= "CREATE TABLE products (code INTEGER PRIMARY KEY, product_name TEXT, countries_en TEXT, serving_size float, energy_100g float, nutrition_score_uk_100g int);"
query_nutrient= "CREATE TABLE nutrient (code INTEGER PRIMARY KEY, product_name TEXT, proteins_100g TEXT, fat_100g float, cholesterol_100g float, carbohydrates_100g float, sugars_100g float);"

In [13]:
# open a Cursor
cursor = db.cursor()

In [14]:
#cursor.execute("DROP TABLE products;")
#cursor.execute("DROP TABLE nutrient;")
db.commit()

In [15]:
cursor.execute(query_product)
cursor.execute(query_nutrient)
db.commit()

In [16]:
# check the tables
cursor.execute("SELECT name FROM sqlite_master where type='table';")
results = cursor.fetchall()
print(results)

[('products',), ('nutrient',)]


**Loading Data from DataFram to Database**

In [17]:
df_sample = df.sample(n=1000)
df_sample[columns_product].to_sql(name='products', con=db, if_exists='replace', index=False)
df_sample[columns_nutrient].to_sql(name='nutrient', con=db, if_exists='replace', index=False)

**Running a Test Query**

In [18]:
def run_query(query):
    return pd.read_sql_query(query, db)

In [19]:
run_query("SELECT * from products limit 10;")

,code,product_name,countries_en,serving_size,energy_100g,nutrition_score_uk_100g
0,26122762,Creamy Aoli,Australia,None,NaN,NaN
1,20186258,Stir fry noodles,Belgium,None,NaN,NaN
2,8712566221059,Soupe Tomate Céleri Ligne,France,None,84.0,1.0
3,3392460482012,"Forme+ (5 Vitamines & 4 Minéraux), Céréale Com...",France,"1 biscotte (8,20 g)",1625.0,7.0
4,3266980931551,None,France,None,NaN,NaN
5,0078742133225,"Grate Value, Almonds, Sea Salt",United States,28 g (1 ONZ),2540.0,8.0
6,20492915,Manzanas Granny Smith,Spain,None,NaN,NaN
7,0073711415548,"Rye Bread With Caraway Seeds, Restaurant Size",United States,38 g (38 g),1100.0,9.0
8,5060424400362,None,United Kingdom,None,NaN,NaN
9,0087688067761,Spiced Bacon Ends & Pieces,United States,14 g (0.5 ONZ),2092.0,26.0


In [20]:
run_query("SELECT count(*) from products;")

,count(*)
0,1000


In [21]:
run_query("SELECT * from nutrient limit 10;")

,code,product_name,proteins_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g
0,26122762,Creamy Aoli,NaN,NaN,NaN,NaN,NaN
1,20186258,Stir fry noodles,NaN,NaN,NaN,NaN,NaN
2,8712566221059,Soupe Tomate Céleri Ligne,0.70,NaN,NaN,NaN,1.00
3,3392460482012,"Forme+ (5 Vitamines & 4 Minéraux), Céréale Com...",15.00,6.00,NaN,63.00,7.70
4,3266980931551,None,NaN,NaN,NaN,NaN,NaN
5,0078742133225,"Grate Value, Almonds, Sea Salt",21.43,53.57,0.000,17.86,3.57
6,20492915,Manzanas Granny Smith,NaN,NaN,NaN,NaN,NaN
7,0073711415548,"Rye Bread With Caraway Seeds, Restaurant Size",7.89,3.95,0.000,47.37,0.00
8,5060424400362,None,NaN,NaN,NaN,NaN,NaN
9,0087688067761,Spiced Bacon Ends & Pieces,42.86,32.14,0.107,0.00,0.00


In [22]:
run_query("SELECT count(*) from nutrient;")

,count(*)
0,1000


In [23]:
run_query('''select p.product_name, p.nutrition_score_uk_100g, n.proteins_100g, n.fat_100g, n.cholesterol_100g, n.carbohydrates_100g
          from products p inner join nutrient n on p.code=n.code where p.nutrition_score_uk_100g > 0''')

,product_name,nutrition_score_uk_100g,proteins_100g,fat_100g,cholesterol_100g,carbohydrates_100g
0,Soupe Tomate Céleri Ligne,1.0,0.70,NaN,NaN,NaN
1,"Forme+ (5 Vitamines & 4 Minéraux), Céréale Com...",7.0,15.00,6.00,NaN,63.00
2,"Grate Value, Almonds, Sea Salt",8.0,21.43,53.57,0.000,17.86
3,"Rye Bread With Caraway Seeds, Restaurant Size",9.0,7.89,3.95,0.000,47.37
4,Spiced Bacon Ends & Pieces,26.0,42.86,32.14,0.107,0.00
5,Smoked Pork Sausage,20.0,13.79,24.14,0.043,1.72
6,Mediterranean Apricots,8.0,2.70,0.00,0.000,62.16
7,Potatoes With Cheese Sauce Mix.,9.0,7.14,0.00,0.000,78.57
8,Gluten Free Mac & Cheese,5.0,7.93,10.13,0.031,26.87
9,Pain Gris Pur Froment,13.0,8.05,NaN,NaN,NaN


**Randomly selecting 1000 rows from our DataFrame**